In [2]:
!pip install google.generativeai

   ---------------------------------------- 146.8/146.8 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 664.5/664.5 kB 7.0 MB/s eta 0:00:00
   ---------------------------------------- 138.3/138.3 kB 8.5 MB/s eta 0:00:00
   ---------------------------------------- 12.7/12.7 MB 11.5 MB/s eta 0:00:00
   --------------------------------------- 229.1/229.1 kB 13.7 MB/s eta 0:00:00
   ---------------------------------------- 96.9/96.9 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 48.8/48.8 kB ? eta 0:00:00
   --------------------------------------- 413.4/413.4 kB 12.6 MB/s eta 0:00:00
   ---------------------------------------- 3.8/3.8 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.59.2
    Uninstalling grpcio-1.59.2:
      Successfully uni

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.25.3 which is incompatible.
tensorboardx 2.6 requires protobuf<4,>=3.8.0, but you have protobuf 4.25.3 which is incompatible.


In [ ]:
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import pathlib
import textwrap
import requests
import os
import json
import pandas as pd
import numpy as np
import time

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
# from google.colab import userdata

import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer